In [1]:
import numpy as np
from fda import *
import pickle
from federatedAlgs import *
from dataGenerator import *
import time
# dataGenerator()

2024-01-16 17:52:33,461	INFO worker.py:1724 -- Started a local Ray instance.


In [5]:
# number of basis functions in basis system
t = 20
rangeval = [0, 100]
basisobj = create_bspline_basis(rangeval, t)
# number of observations pre local server n
n = 100
# number of features p 20
p = 20
betaPar = fdPar(basisobj, 0, 0)
# number of observations pre local server n
samplesPerWorker = 100
# number of features p 20
numPredictors = 20
yfine = np.linspace(1, 100, 100)
allsamples = set([i for i in range(100)])
numworkersseq = [2, 4, 6, 8]
bbspl2 = bifd(np.linspace(1, pow(t, 2), pow(t, 2)).reshape((t, t)), create_bspline_basis(rangeval, t),
              create_bspline_basis(rangeval, t))
bifdbasis = bifdPar(bbspl2, 0, 0, 0, 0)
betaList = [betaPar, bifdbasis]
time1 = np.zeros([len(numworkersseq), 4])
time2 = np.zeros([len(numworkersseq), 4])

In [6]:
# orginal time: Number of local servers : 2; 0 fold.
# 1 1705387323.2915132
# 2 1705387370.324656
# 3 1705387370.324656
# 4 1705387373.236033
# LSA Round 2 : 14 selected.
# 5 1705387376.2423809
# 6 1705387376.2455325

# ray time:
# 1 1705388451.6095245
# 2 1705388456.0331485

In [7]:
for numworkers in numworkersseq:
    # 4-fold CV
    for hh in range(4):
        print(f'Number of local servers : {numworkers}; {hh} fold.')
        test = set([i for i in range(hh * 25, (hh + 1) * 25)])
        train = list(allsamples - test)
        test = setGenerator(test, samplesPerWorker, numworkers)
        predictors = np.zeros([len(yfine), samplesPerWorker * numworkers, numPredictors])
        response = np.zeros([len(yfine), samplesPerWorker * numworkers])
        x = []
        y = []
        for l in range(1, numworkers + 1):
            with open('tmp/yfdobj_' + str(l) + '_' + str(numworkers), 'rb') as file:
                yfdobj = pickle.load(file)
            with open('tmp/predictorLst_' + str(l) + '_' + str(numworkers), 'rb') as file:
                predictorLst = pickle.load(file)
            xfdobjTrainLst = []
            for i in range(numPredictors):
                predictors[:, (samplesPerWorker * (l - 1)): (samplesPerWorker * l), i] = eval_fd(yfine,
                                                                                                 predictorLst[i])
                temp1 = smooth_basis(yfine, predictors[:, [i + samplesPerWorker * (l - 1) - 1 for i in train], i],
                                     basisobj).fd
                xfdobjTrainLst.append(temp1)
            x.append(xfdobjTrainLst)
            response[:, (samplesPerWorker * (l - 1)): (samplesPerWorker * l)] = eval_fd(yfine, yfdobj)
            responsefdobjTrain = smooth_basis(yfine,
                                              response[:, [i + samplesPerWorker * (l - 1) - 1 for i in train]],
                                              basisobj).fd
            y.append(responsefdobjTrain)

        yfdobjTest = smooth_basis(yfine, response[:, test], basisobj).fd
        responseTest = eval_fd(yfine, yfdobjTest)
        boost_control = 10
        step_length = 0.5
        lin = linmod(x[0][1], y[0], betaList)
        step_length = 0.5
        start = time.time()
        federatedFunctionalGradBoostLSA(x, y, betaList, boost_control, step_length)
        end = time.time()
        time1[numworkersseq.index(numworkers), hh] = end - start
        start = time.time()
        federatedFunctionalGradBoostAvg(x, y, betaList, boost_control, step_length)
        end = time.time()
        time2[numworkersseq.index(numworkers), hh] = end - start
print("Comparison of operation times of fed-GB-LSA (LSA) and fed-GB-Average (Avg)")
print('operation times of fed-GB-LSA (LSA)')
print(np.mean(time1, 1))
print('operation times of fed-GB-Average (Avg)')
print(np.mean(time2, 1))

Number of local servers : 2; 0 fold.
LSA Round 2 : 9 selected.
LSA Round 3 : 9 selected.
LSA Round 4 : 8 selected.
LSA Round 5 : 9 selected.
LSA Round 6 : 9 selected.
LSA Round 7 : 11 selected.
LSA Round 8 : 8 selected.
LSA Round 9 : 8 selected.
LSA Round 10 : 2 selected.
AVG Round 2 : 0 selected.
AVG Round 3 : 0 selected.
AVG Round 4 : 0 selected.
AVG Round 5 : 0 selected.
AVG Round 6 : 0 selected.
AVG Round 7 : 1 selected.
AVG Round 8 : 0 selected.
AVG Round 9 : 0 selected.
AVG Round 10 : 1 selected.
Number of local servers : 2; 1 fold.
LSA Round 2 : 12 selected.
LSA Round 3 : 12 selected.
LSA Round 4 : 2 selected.
LSA Round 5 : 2 selected.
LSA Round 6 : 2 selected.
LSA Round 7 : 7 selected.
LSA Round 8 : 11 selected.
LSA Round 9 : 2 selected.
LSA Round 10 : 2 selected.
AVG Round 2 : 0 selected.
AVG Round 3 : 0 selected.
AVG Round 4 : 0 selected.
AVG Round 5 : 0 selected.
AVG Round 6 : 0 selected.
AVG Round 7 : 1 selected.
AVG Round 8 : 0 selected.
AVG Round 9 : 0 selected.
AVG Round

In [1]:
import numpy as np
from fda import *
import pickle
from federatedAlgs import *
from dataGenerator import *
import time
import sys
from tqdm import tqdm

# number of basis functions in basis system
t = 20
rangeval = [0, 100]
basisobj = create_bspline_basis(rangeval, t)
# number of observations pre local server n
n = 100
# number of features p 20
p = 20
betaPar = fdPar(basisobj, 0, 0)
# number of observations pre local server n
samplesPerWorker = 100
# number of features p 20
numPredictors = 20
yfine = np.linspace(1, 100, 100)
allsamples = set([i for i in range(100)])
numworkersseq = [2, 4, 6, 8, 10]
bbspl2 = bifd(np.linspace(1, pow(t, 2), pow(t, 2)).reshape((t, t)), create_bspline_basis(rangeval, t),
              create_bspline_basis(rangeval, t))
bifdbasis = bifdPar(bbspl2, 0, 0, 0, 0)
betaList = [betaPar, bifdbasis]
time1 = np.zeros([len(numworkersseq), 4])
trans_time1 = np.zeros([len(numworkersseq), 4])
sub_trans_time1 = np.zeros([len(numworkersseq), 4])
trans_data1 = np.zeros([len(numworkersseq), 4])
time2 = np.zeros([len(numworkersseq), 4])
trans_time2 = np.zeros([len(numworkersseq), 4])
sub_trans_time2 = np.zeros([len(numworkersseq), 4])
trans_data2 = np.zeros([len(numworkersseq), 4])

print(time.time())

log_path = "log/" + str(time.time()) + ".txt"

for numworkers in tqdm(numworkersseq):
    # 4-fold CV
    for hh in range(4):
        with open(log_path, "a+") as fw:
            fw.write(f'Number of local servers : {numworkers}; {hh} fold.\n')

        test = set([i for i in range(hh * 25, (hh + 1) * 25)])
        train = list(allsamples - test)
        test = setGenerator(test, samplesPerWorker, numworkers)
        predictors = np.zeros([len(yfine), samplesPerWorker * numworkers, numPredictors])
        response = np.zeros([len(yfine), samplesPerWorker * numworkers])
        x = []
        y = []
        for l in range(1, numworkers + 1):
            with open('tmp/yfdobj_' + str(l) + '_' + str(numworkers), 'rb') as file:
                yfdobj = pickle.load(file)
            with open('tmp/predictorLst_' + str(l) + '_' + str(numworkers), 'rb') as file:
                predictorLst = pickle.load(file)
            xfdobjTrainLst = []
            for i in range(numPredictors):
                predictors[:, (samplesPerWorker * (l - 1)): (samplesPerWorker * l), i] = eval_fd(yfine,
                                                                                                 predictorLst[i])
                temp1 = smooth_basis(yfine, predictors[:, [i + samplesPerWorker * (l - 1) - 1 for i in train], i],
                                     basisobj).fd
                xfdobjTrainLst.append(temp1)
            x.append(xfdobjTrainLst)
            response[:, (samplesPerWorker * (l - 1)): (samplesPerWorker * l)] = eval_fd(yfine, yfdobj)
            responsefdobjTrain = smooth_basis(yfine,
                                              response[:, [i + samplesPerWorker * (l - 1) - 1 for i in train]],
                                              basisobj).fd
            y.append(responsefdobjTrain)

        yfdobjTest = smooth_basis(yfine, response[:, test], basisobj).fd
        responseTest = eval_fd(yfine, yfdobjTest)
        boost_control = 10
        step_length = 0.5
        lin = linmod(x[0][1], y[0], betaList)
        step_length = 0.5
        start = time.time()
        test1_status = federatedFunctionalGradBoostLSA(x, y, betaList, boost_control, step_length)
        end = time.time()
        time1[numworkersseq.index(numworkers), hh] = end - start
        trans_time1[numworkersseq.index(numworkers), hh] = test1_status[1]
        # sub_trans_time1[numworkersseq.index(numworkers), hh] = test1_status[3]
        trans_data1[numworkersseq.index(numworkers), hh] = test1_status[2]
        start = time.time()
        test2_status = federatedFunctionalGradBoostAvg(x, y, betaList, boost_control, step_length)
        end = time.time()
        time2[numworkersseq.index(numworkers), hh] = end - start
        trans_time2[numworkersseq.index(numworkers), hh] = test2_status[1]
        # sub_trans_time2[numworkersseq.index(numworkers), hh] = test2_status[3]
        trans_data2[numworkersseq.index(numworkers), hh] = test2_status[2]

with open(log_path, "a+") as fw:
    fw.write("Comparison of operation times of fed-GB-LSA (LSA) and fed-GB-Average (Avg)\n")
    fw.write('operation times of fed-GB-LSA (LSA)\n')
    fw.write(str(np.mean(time1, 1)))
    fw.write('\noperation times of fed-GB-Average (Avg)\n')
    fw.write(str(np.mean(time2, 1)))
    fw.write('\nAverage transmission times of fed-GB-LSA (LSA)\n')
    fw.write(str(np.mean(trans_time1, 1)))
    # fw.write(str(np.mean(sub_trans_time1, 1)))
    fw.write('\nAverage transmission times of fed-GB-Average (Avg)\n')
    fw.write(str(np.mean(trans_time2, 1)))
    # fw.write(str(np.mean(sub_trans_time2, 1)))
    fw.write('\nTotal transmission data (bytes) of fed-GB-LSA (LSA)\n')
    fw.write(str(np.mean(trans_data1, 1)))
    fw.write('\nTotal transmission data (bytes) of fed-GB-Average (Avg)\n')
    fw.write(str(np.mean(trans_data2, 1)))

2024-02-06 18:49:37,391	INFO worker.py:1724 -- Started a local Ray instance.


1707263378.6098225


  0%|          | 0/5 [00:00<?, ?it/s]

LSA Round 2 : 9 selected.
LSA Round 3 : 9 selected.
LSA Round 4 : 2 selected.
LSA Round 5 : 8 selected.
LSA Round 6 : 9 selected.
LSA Round 7 : 9 selected.
LSA Round 8 : 2 selected.
LSA Round 9 : 9 selected.
LSA Round 10 : 9 selected.
AVG Round 2 : 0 selected.
AVG Round 3 : 0 selected.
AVG Round 4 : 0 selected.
AVG Round 5 : 0 selected.
AVG Round 6 : 0 selected.
AVG Round 7 : 1 selected.
AVG Round 8 : 0 selected.
AVG Round 9 : 0 selected.
AVG Round 10 : 1 selected.
LSA Round 2 : 12 selected.
LSA Round 3 : 2 selected.
LSA Round 4 : 2 selected.
LSA Round 5 : 2 selected.
LSA Round 6 : 2 selected.
LSA Round 7 : 2 selected.
LSA Round 8 : 11 selected.
LSA Round 9 : 2 selected.
LSA Round 10 : 2 selected.
AVG Round 2 : 0 selected.
AVG Round 3 : 0 selected.
AVG Round 4 : 0 selected.
AVG Round 5 : 0 selected.
AVG Round 6 : 0 selected.
AVG Round 7 : 1 selected.
AVG Round 8 : 0 selected.
AVG Round 9 : 0 selected.
AVG Round 10 : 1 selected.
LSA Round 2 : 12 selected.
LSA Round 3 : 2 selected.
LSA R

 20%|██        | 1/5 [06:23<25:33, 383.43s/it]

...

AVG Round 10 : 0 selected.
LSA Round 2 : 10 selected.
LSA Round 3 : 18 selected.
LSA Round 4 : 2 selected.
LSA Round 5 : 18 selected.
LSA Round 6 : 18 selected.
LSA Round 7 : 18 selected.
LSA Round 8 : 18 selected.
LSA Round 9 : 18 selected.
LSA Round 10 : 14 selected.
AVG Round 2 : 0 selected.
AVG Round 3 : 2 selected.
AVG Round 4 : 2 selected.
AVG Round 5 : 1 selected.
AVG Round 6 : 1 selected.
AVG Round 7 : 1 selected.
AVG Round 8 : 1 selected.
AVG Round 9 : 1 selected.
AVG Round 10 : 1 selected.
LSA Round 2 : 14 selected.
LSA Round 3 : 8 selected.
LSA Round 4 : 8 selected.
LSA Round 5 : 8 selected.
LSA Round 6 : 8 selected.
LSA Round 7 : 8 selected.
LSA Round 8 : 12 selected.
LSA Round 9 : 8 selected.
LSA Round 10 : 8 selected.
AVG Round 2 : 0 selected.
AVG Round 3 : 2 selected.
AVG Round 4 : 1 selected.
AVG Round 5 : 2 selected.
AVG Round 6 : 1 selected.
AVG Round 7 : 1 selected.
AVG Round 8 : 2 selected.
AVG Round 9 : 1 selected.
AVG Round 10 : 1 selected.
LSA Round 2 : 4 selecte

 80%|████████  | 4/5 [38:47<10:41, 641.93s/it]

AVG Round 10 : 1 selected.
LSA Round 2 : 1 selected.
LSA Round 3 : 5 selected.
LSA Round 4 : 9 selected.
LSA Round 5 : 12 selected.
LSA Round 6 : 9 selected.
LSA Round 7 : 1 selected.
LSA Round 8 : 9 selected.
LSA Round 9 : 9 selected.
LSA Round 10 : 1 selected.
AVG Round 2 : 1 selected.
AVG Round 3 : 0 selected.
AVG Round 4 : 0 selected.
AVG Round 5 : 0 selected.
AVG Round 6 : 1 selected.
AVG Round 7 : 0 selected.
AVG Round 8 : 0 selected.
AVG Round 9 : 1 selected.
AVG Round 10 : 0 selected.
LSA Round 2 : 2 selected.
LSA Round 3 : 6 selected.
LSA Round 4 : 10 selected.
LSA Round 5 : 10 selected.
LSA Round 6 : 5 selected.
LSA Round 7 : 18 selected.
LSA Round 8 : 12 selected.
LSA Round 9 : 5 selected.
LSA Round 10 : 18 selected.
AVG Round 2 : 1 selected.
AVG Round 3 : 0 selected.
AVG Round 4 : 0 selected.
AVG Round 5 : 1 selected.
AVG Round 6 : 0 selected.
AVG Round 7 : 2 selected.
AVG Round 8 : 0 selected.
AVG Round 9 : 1 selected.
AVG Round 10 : 0 selected.
LSA Round 2 : 2 selected.
L

100%|██████████| 5/5 [53:10<00:00, 638.04s/it]

AVG Round 10 : 0 selected.
